# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file=os.path.join('..','WeatherPy','output_data','Clean_City_data.csv')
df=pd.read_csv(file)
del df['Unnamed: 0']
df.head()


,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed,Country,Date
0,port alfred,-33.59,26.89,57.99,90.0,70.0,6.89,ZA,1.592235e+09
1,avarua,-21.21,-159.78,69.80,73.0,75.0,13.87,CK,1.592235e+09
2,busselton,-33.65,115.33,56.19,73.0,99.0,5.37,AU,1.592235e+09
3,blagoyevo,63.37,47.92,74.89,46.0,91.0,7.00,RU,1.592235e+09
4,eisenberg,50.97,11.90,71.65,79.0,100.0,1.99,DE,1.592235e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]
humidity=df["Humidity (%)"]

# Plot Heatmap
fig = gmaps.figure(map_type='TERRAIN', center=(0, 0), zoom_level=2,layout={
        'width': '1000px',
        'height': '700px',
        'padding': '3px',
        'border': '1px solid black'
})

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)


# Display figure
fig



Figure(layout=FigureLayout(border='1px solid black', height='700px', padding='3px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_df=df.loc[(df["Max Temp (F)"]>=70) & (df["Max Temp (F)"]<=80) & (df["Wind Speed"]<=10) & (df["Cloudiness (%)"]==0),:]
ideal_df=ideal_df.reset_index(drop=True)
ideal_df.head(20)

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed,Country,Date
0,henties bay,-22.12,14.28,70.00,53.0,0.0,7.00,NaN,1.592235e+09
1,pingzhuang,42.04,119.29,70.77,27.0,0.0,2.30,CN,1.592235e+09
2,shakawe,-18.37,21.85,76.68,26.0,0.0,5.82,BW,1.592235e+09
3,suoyarvi,62.08,32.35,73.22,35.0,0.0,6.67,RU,1.592235e+09
4,saint-michel-des-saints,46.68,-73.92,72.19,32.0,0.0,1.70,CA,1.592235e+09
5,lavumisa,-27.31,31.89,71.60,28.0,0.0,4.70,SZ,1.592235e+09
6,shihezi,44.30,86.03,79.75,21.0,0.0,9.84,CN,1.592235e+09
7,bardiyah,31.76,25.09,79.48,46.0,0.0,8.93,LY,1.592235e+09
8,margate,51.38,1.39,71.01,63.0,0.0,1.01,GB,1.592235e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotels=[]

# Loop through dataframe to get nearest hotel
for city in range(len(ideal_df["City"])):
    lat = ideal_df.loc[city]["Lat"]
    lng = ideal_df.loc[city]["Lng"]
    city_coords = f"{lat},{lng}"
    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
              }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    hotel_response = requests.get(base_url, params=params).json()
 
    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
ideal_df["Nearest Hotel"] = hotels
ideal_df.head(10)



,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed,Country,Date,Nearest Hotel
0,henties bay,-22.12,14.28,70.00,53.0,0.0,7.00,NaN,1.592235e+09,Fishermans Guest House
1,pingzhuang,42.04,119.29,70.77,27.0,0.0,2.30,CN,1.592235e+09,Haoke Guest House
2,shakawe,-18.37,21.85,76.68,26.0,0.0,5.82,BW,1.592235e+09,Ditlhapi Guest House
3,suoyarvi,62.08,32.35,73.22,35.0,0.0,6.67,RU,1.592235e+09,Mini-Hotel Comfort
4,saint-michel-des-saints,46.68,-73.92,72.19,32.0,0.0,1.70,CA,1.592235e+09,Motel Du Mont trinity
5,lavumisa,-27.31,31.89,71.60,28.0,0.0,4.70,SZ,1.592235e+09,ROADSIDE B&B
6,shihezi,44.30,86.03,79.75,21.0,0.0,9.84,CN,1.592235e+09,Home Inn Shihezi Dongwu Road Century Square
7,bardiyah,31.76,25.09,79.48,46.0,0.0,8.93,LY,1.592235e+09,بورتو بردية
8,margate,51.38,1.39,71.01,63.0,0.0,1.01,GB,1.592235e+09,Premier Inn Margate hotel


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{ideal_df['Nearest Hotel']}</dd>
<dt>City</dt><dd>{ideal_df['City']}</dd>
<dt>Country</dt><dd>{ideal_df['Country']}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Lat", "Lng"]]


In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', padding='3px', width='1000px'))